In [1]:
print('hello')

hello


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import random
import mmap
import intel_extension_for_pytorch as ipex
print(torch.__version__)
device = 'xpu' if torch.xpu.is_available() else 'cpu'
print(f'Using device: {device}')

2.8.0+xpu
Using device: xpu


In [3]:
block=64
batch=128
max_iters=3000
# eval_interval=500
eval_iters=500
learning_rate=3e-3
n_embd = 384
n_layer = 8
n_head = 8
dropout = 0.2

In [ ]:
with open('vocab.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))
print(text[:200])
print(f'Total characters: {chars}')
vocab_size = len(chars)

Letter 1

_To Mrs. Saville, England._


St. Petersburgh, Dec. 11th, 17—.


You will rejoice to hear that no disaster has accompanied the
commencement of an enterprise which you have regarded with suc
Total characters: ['\n', ' ', '!', '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'Y', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'æ', 'è', 'é', 'ê', 'ô', '—', '‘', '’', '“', '”', '\ufeff']


In [5]:
stringtoint={ch:i for i,ch in enumerate(chars)}
inttostring={i:ch for i,ch in enumerate(chars)}
encode=lambda s: [stringtoint[c] for c in s]
decode=lambda l: ''.join([inttostring[i] for i in l])

print(encode('hello world'))
print(decode(encode('hello world')))

[54, 51, 58, 58, 61, 1, 69, 61, 64, 58, 50]
hello world


In [ ]:
# data = torch.tensor(encode(text), dtype=torch.long)
# print(data.shape, data.dtype)
# print(data[:100])

torch.Size([418920]) torch.int64
tensor([83, 32, 51, 66, 66, 51, 64,  1,  9,  0,  0, 46, 39, 61,  1, 33, 64, 65,
         7,  1, 38, 47, 68, 55, 58, 58, 51,  5,  1, 25, 60, 53, 58, 47, 60, 50,
         7, 46,  0,  0,  0, 38, 66,  7,  1, 36, 51, 66, 51, 64, 65, 48, 67, 64,
        53, 54,  5,  1, 24, 51, 49,  7,  1,  9,  9, 66, 54,  5,  1,  9, 15, 78,
         7,  0,  0,  0, 43, 61, 67,  1, 69, 55, 58, 58,  1, 64, 51, 56, 61, 55,
        49, 51,  1, 66, 61,  1, 54, 51, 47, 64])


In [ ]:
def get_random_chunk(split):
    filename = 'output_train.txt' if split == 'train' else 'output_val.txt'
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            file_size = len(mm)
            start_idx = random.randint(0, (file_size)-block*batch)
            
            mm.seek(start_idx)
            chunk = mm.read(block*batch-1)
            decoded_chunk = chunk.decode('utf-8', errors='ignore').replace('\r', '')
            data = torch.tensor(encode(decoded_chunk), dtype=torch.long)
    return data

In [ ]:
# n=int(0.8*len(data))
# train_data = data[:n]
# val_data = data[n:]
# print(train_data.shape, val_data.shape)

def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block,(batch,))
    # print(ix)
    x = torch.stack([data[i:i+block] for i in ix])
    y = torch.stack([data[i+1:i+block+1] for i in ix])
    x=x.to(device); y=y.to(device)
    return x,y 

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)   

torch.Size([335136]) torch.Size([83784])
inputs:
tensor([[ 1, 48, 71,  ..., 60, 51, 65],
        [65,  1, 52,  ...,  1, 51, 60],
        [55, 72, 61,  ..., 65,  1, 61],
        ...,
        [61, 52,  1,  ..., 55, 66, 54],
        [60, 61, 66,  ..., 55, 59, 62],
        [50,  1, 54,  ..., 55, 58, 61]], device='xpu:0')
targets:
tensor([[48, 71,  1,  ..., 51, 65, 65],
        [ 1, 52, 55,  ..., 51, 60, 49],
        [72, 61, 60,  ...,  1, 61, 64],
        ...,
        [52,  1, 66,  ..., 66, 54,  1],
        [61, 66,  1,  ..., 59, 62, 51],
        [ 1, 54, 51,  ..., 58, 61, 65]], device='xpu:0')


In [8]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block, block)))
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

In [9]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for i in range(num_heads)])
        self.proj = nn.Linear(head_size*num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [10]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )
        
    def forward(self, x):
        return self.net(x)

In [11]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x+y)
        y = self.ffwd(x)
        x = self.ln2(x+y)
        return x

In [ ]:
class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for i in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        # self.apply(self._init_weights)
        
        def _init_weights(self, module):
            if isinstance(module, nn.Linear):
                torch.nn.init.normal_(model.weight, mean=0.0, std=0.02)
                if module.bias is not None:
                    torch.nn.init.zeros_(module.bias)
                elif isinstance(module, nn.Embedding):
                    torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        
        
    def forward(self, idx, targets=None):
        # logits = self.token_embedding_table(idx)
        B,T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets is None:
            loss=None
        else:
            B, T, C = logits.shape
            logits=logits.view(B*T,C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for i in range(max_new_tokens):
            idx_cond = idx[:, -block:]
            logits, loss = self.forward(idx_cond)
            logits = logits[:, -1, :]
            probs =F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=-1)
        return idx
    
model = GPTLanguageModel(vocab_size)
print('Model instance created.')
MODEL_PATH = 'gpt1_model.pth'

# --- New Loading Block ---
try:
    # Load the saved state_dict
    # We use map_location to ensure the model loads correctly
    # onto the device you've chosen (e.g., 'xpu' or 'cpu').
    model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
    print('Model weights loaded.')
except FileNotFoundError:
    print('No saved model found. Starting from scratch.')
# -------------------------

m = model.to(device)


Iê“S;lcFle0O0Sækq,3  s!m”pgmPO]oTj(B:cl6:K!!ybw—‘8ObxG]êuJ5;8æ”yl,7yr(lLMcFwLLfpe1æTA”’6_sFqTfCDF)H“CvJMh‘roj‘Mé;:n
êP?rTw68RæELDiUY(L5NB]‘-”Jv—é;46V!L2)RyWæCVæwé)(E8N4Bé?bC?FêSèUNcghqCMD9:nEOx:t
é(F8(lyAVq!,6ESvPFM95GGrNær,
9PyrJ7;g7uN?lEUFèo?bèc1[ôb;”,é— sFyA17“(l3HADvC ;wha(?;wk0euRrR np5’DUhUC;U,(W2
;trBô6An6(M:k.p;xjCeFôgMxzGbbp1r“x)x GCB]uxyE?—eH( ]2f6’“[L
n[l1;Fqhw?NePmJlFM2— ixo-x2c),æUgUrô1,ceNcm]zWB[x“é8n.Le 6HV(dAnAq.S9m(G.(NL“C)4.;F6ezl4HKdFG;FnPl(p“Vé)nYMIrBu _)HN5é“ô7dD“nmyUo)Ow(éê


In [13]:
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ['train','val']:
        losses=torch.zeros(eval_iters)
        for i in range(eval_iters):
            X,Y=get_batch(split)
            logits, loss = model(X, Y)
            losses[i]=loss.item()
        out[split]=losses.mean()
    model.train()
    return out

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iteration in range(max_iters):
    if iteration%eval_iters==0:
        losses=estimate_loss()
        print(f"step {iteration}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    xb,yb=get_batch('train')
    
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
# print(loss.item())
MODEL_PATH = 'gpt1_model.pth'
torch.save(model.state_dict(), MODEL_PATH)

step 0: train loss 4.5882, val loss 4.5895
step 500: train loss 3.0537, val loss 3.0494
step 1000: train loss 3.0521, val loss 3.0483
step 1500: train loss 3.0527, val loss 3.0473
step 2000: train loss 3.0513, val loss 3.0468
step 2500: train loss 3.0441, val loss 3.0443
